## Introduction to Optimisation with Pyomo

### Installing and Importing Packages 

We first need to pull in all the packages we will be using. Pyomo is a Python-based, open-source optimization modelling language with a diverse set of optimization capabilities. For more information, see the Pyomo [documentation](https://pyomo.readthedocs.io/en/stable/).

In [9]:
# Only run once at the start
!pip install -q pyomo
!chmod +x 'ipopt'

'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import matplotlib.pyplot as plt
from pyomo.environ import *
import numpy as np
from ipywidgets import FloatSlider, interact

In [22]:
# Create a model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=Reals)
model.x2 = Var(domain=Reals)

# Objective function
model.obj = Objective(expr = model.x1**2 + model.x2**2, sense=minimize)

# Create a solver
solver = SolverFactory('ipopt', executable="ipopt")

# Solve the model
solver.solve(model)

# Display the results
print(f"x1: {model.x1()}")
print(f"x2: {model.x2()}")

model.pprint()

ApplicationError: Could not execute the command: ['c:\\Users\\eep19\\Documents\\sc_opt_course\\ipopt', 'C:\\Users\\eep19\\AppData\\Local\\Temp\\tmp1hrg4qp7.pyomo.nl', '-AMPL']	Error message: [WinError 193] %1 is not a valid Win32 application

In [ ]:
# Create a grid of x1 and x2 values
x1_vals = np.linspace(-5, 5, 400)
x2_vals = np.linspace(-5, 5, 400)
X1, X2 = np.meshgrid(x1_vals, x2_vals)

# Define the objective function x1^2 + x2^2
Z = X1**2 + X2**2

# Create a contour plot
plt.figure(figsize=(5, 4))
contour = plt.contour(X1, X2, Z, levels=30, cmap='Blues', alpha=0.7)
plt.colorbar(contour)

# Highlight the minimum point
plt.plot( model.x1(), model.x2(), 'bx', markersize=7, label=f'Solution at ({model.x1():.2f}, {model.x2():.2f})')

# Set plot labels and title
plt.xlabel('$x_1$', fontsize=12)
plt.ylabel('$x_2$', fontsize=12)
plt.title('Contour Plot of $x_1^2 + x_2^2$', fontsize=14)

# Add a legend
plt.legend()

# Display the plot
plt.grid(True)

In [4]:
# Create a model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=Reals)
model.x2 = Var(domain=Reals)

# Objective function
model.obj = Objective(expr = model.x1**2 + model.x2**2, sense=minimize)

# Add an Equality constraint
model.constr = Constraint(expr=model.x1 + model.x2 >=1)

# Create a solver
solver = SolverFactory('ipopt', executable='ipopt')

# Solve the model
solver.solve(model)

# Display the results
print(f"x1: {model.x1()}")
print(f"x2: {model.x2()}")

model.pprint()

for solver ipopt. File with name=ipopt either does not exist or it is not
executable. To skip this validation, call set_executable with validate=False.
Traceback (most recent call last):
  File "c:\Users\eep19\AppData\Local\anaconda3\Lib\site-packages\pyomo\opt\base\solvers.py", line 148, in __call__
    opt = self._cls[_name](**kwds)
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\eep19\AppData\Local\anaconda3\Lib\site-packages\pyomo\solvers\plugins\solvers\IPOPT.py", line 40, in __init__
    super(IPOPT, self).__init__(**kwds)
  File "c:\Users\eep19\AppData\Local\anaconda3\Lib\site-packages\pyomo\opt\solver\shellcmd.py", line 65, in __init__
    self.set_executable(name=executable, validate=validate)
  File "c:\Users\eep19\AppData\Local\anaconda3\Lib\site-packages\pyomo\opt\solver\shellcmd.py", line 114, in set_executable
    raise ValueError(
ValueError: Failed to set executable for solver ipopt. File with name=ipopt either does not exist or it is not executable. To skip this va

RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "ipopt"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: ipopt
	type: ipopt
	_args: ()
	options: {}

In [ ]:
x1_vals = np.linspace(-5, 5, 400)
x2_vals = np.linspace(-5, 5, 400)
X1, X2 = np.meshgrid(x1_vals, x2_vals)

# Define the objective function x1^2 + x2^2
Z = X1**2 + X2**2

# Create a contour plot
plt.figure(figsize=(5, 4))
contour = plt.contour(X1, X2, Z, levels=30, cmap='Blues', alpha=0.7)
plt.colorbar(contour)

# Plot Feasible Region
plt.fill_betweenx(x1_vals, (1-x1_vals), 5, where=((1-x1_vals) <= 5), color='lightgreen', alpha=0.3, label='$x_1 + x_2 \geq 1$')
plt.plot(x1_vals, (1-x1_vals), color='black', linestyle='--', linewidth=1,)

# Highlight Solution
plt.plot( model.x1(), model.x2(), 'bx', markersize=7, label=f'Solution at ({model.x1():.2f}, {model.x2():.2f})')

# Labels & Titles
plt.xlabel('$x_1$', fontsize=12)
plt.ylabel('$x_2$', fontsize=12)
plt.title(f'Contour Plot with Feasible Region', fontsize=14)

# Set plot limits
plt.xlim(-5, 5)
plt.ylim(-5, 5)

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Create a model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=Reals)
model.x2 = Var(domain=Reals)

# Objective function
model.obj = Objective(expr = model.x1**2 + model.x2**2, sense=minimize)

# Add an inequality constraint
model.constr2 = Constraint(expr=model.x2 + model.x1 >= 1)

# Add an equality constraint
model.constr1 = Constraint(expr=model.x2 == 1)

# Create a solver
solver = SolverFactory('ipopt', executable='/content/drive/MyDrive/ColabNotebooks/ipopt')

# Solve the model
solver.solve(model)

# Display the results
print(f"x1: {model.x1()}")
print(f"x2: {model.x2()}")

model.pprint()

In [ ]:
x1_vals = np.linspace(-5, 5, 400)
x2_vals = np.linspace(-5, 5, 400)
X1, X2 = np.meshgrid(x1_vals, x2_vals)

# Define the objective function x1^2 + x2^2
Z = X1**2 + X2**2

# Create a contour plot
plt.figure(figsize=(5, 4))
contour = plt.contour(X1, X2, Z, levels=30, cmap='Blues', alpha=0.7)
plt.colorbar(contour)

# Define the boundaries
x2_boundary1 = 1 - x1_vals  # x1 + x2 = 1 -> x2 = 1 - x1
x2_boundary2 = 5 * np.ones_like(x1_vals)  # x2 = 2
x2_equal_x1 = x1_vals  # x1 = x2

# Fill the area between the lines for x1 + x2 >= 1 and x2 = 2
plt.fill_between(x1_vals, np.maximum(x2_boundary1, 1), x2_boundary2,
                  where=(x2_boundary1 <= x2_boundary2),
                  color='lightgreen', alpha=0.3,
                  label='$x_1 + x_2 \\geq 1$ and $x_2 = 2$')


# Add a vertical line at the constraint x1 = -2
plt.axhline(1, color='black', linewidth=1, label='$x_1 = 1$ (Constraint)')
plt.plot(x1_vals, (1-x1_vals), color='black', linestyle='--', linewidth=1,)

# Highlight Solution
plt.plot(model.x1(), model.x2(), 'bx', markersize=7, label=f'Solution at ({model.x1():.2f}, {model.x2():.2f})')

# Labels & Titles
plt.xlabel('$x_1$', fontsize=12)
plt.ylabel('$x_2$', fontsize=12)
plt.title(f'Contour Plot with Feasible Region', fontsize=14)

# Set plot limits
plt.xlim(-5, 5)
plt.ylim(-5, 5)

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def plot_filled_region(C1=1.0, C2=2.0):
    # Create a grid of x1 values
    x1_vals = np.linspace(-5, 5, 400)

    # Define the boundaries based on the slider values
    x2_boundary1 = C1 - x1_vals  # x1 + x2 = C1 -> x2 = C1 - x1
    x2_boundary2 = C2 * np.ones_like(x1_vals)  # x2 = C2

    # Create a new figure
    plt.figure(figsize=(10, 8))

    # Fill the area that satisfies both constraints
    plt.fill_between(x1_vals,
                     np.maximum(x2_boundary1, C2),  # Fill from the maximum of both boundaries
                     5,  # Upper limit (arbitrary large value)
                     where=(np.maximum(x2_boundary1, C2) <= 5),  # Ensure we fill within the limits
                     color='orange', alpha=0.5,
                     label=f'Feasible Region: $x_1 + x_2 \\geq {C1}$ and $x_2 \\geq {C2}$')

    # Plot the lines for visualization
    plt.plot(x1_vals, x2_boundary1, color='blue', linestyle='-', linewidth=2, label=f'$x_1 + x_2 = {C1}$')
    plt.plot(x1_vals, x2_boundary2, color='red', linestyle='--', linewidth=2, label=f'$x_2 = {C2}$')

    # Set plot limits
    plt.xlim(-5, 5)
    plt.ylim(-5, 5)

    # Set plot labels and title
    plt.xlabel('$x_1$', fontsize=12)
    plt.ylabel('$x_2$', fontsize=12)
    plt.title('Interactive Feasible Region Visualization', fontsize=14)

    # Add a legend
    plt.legend()

    # Display the plot
    plt.grid(True)
    plt.axhline(0, color='black', linewidth=0.5, ls='--')  # X-axis
    plt.axvline(0, color='black', linewidth=0.5, ls='--')  # Y-axis
    plt.show()

# Create interactive sliders for the constraints
C1_slider = FloatSlider(value=1.0, min=0, max=5, step=0.1, description='$C1$ (for $x_1 + x_2 = C1$)')
C2_slider = FloatSlider(value=2.0, min=0, max=5, step=0.1, description='$C2$ (for $x_2 = C2$)')

# Use interact to link the sliders to the plotting function
interact(plot_filled_region, C1=C1_slider, C2=C2_slider);
